# CMPS242 - Homework 3
Yi Ru 1635532, Xiaotong Li 1634362  
October 22, 2017  


## 1. Introduction
In this homework report, we explored and applied the logistic regression and the gradient descent theory, to realize a spam email detector. In section 2, we will introduce some basic mathematical and programming methods used in this homework. In section 3, we will introduce the whole algorithm and realization of the detector, and some relative theories and amendments to the detector, as well as their comparisons with the Batch Gradient Descent. In section 4, we will draw our conclusions. 



## 2. Theory


### 2.1 Logistic Regression
In this case of spam email recognition, we need to apply certain theory in machine learning to classify the data we got. However, the linear regression model we used previously can not guarantee valid classification when the data amount is so large. So we need to introduce the logistic regression model to solve this, whose mathematical form is as follows: 
$$ h(z)=\frac{1}{1+e^{-z}}\tag{2.1}$$
Through applying this function, we can map a linear distribution $z$ to a probability distribution where $h(z)$ is bounded by 0 and 1. So we can use this model to classify the data validly as we want to. 


### 2.2 Cost Function
To evaluate the level of the classification resulted from parameter fitting, we introduce a cost function using cross entropy:
$$ J(\omega)=\frac{1}{M}\sum_{i=1}^{M} \left( y_i \text{log}\hat{y_i} + (1-y_i) \text{log}(1-\hat{y_i}) \right)
\tag{2.2}$$
Where $J(\omega)$ represents the cost of the whole model, $\omega$ represents the current parameter, $M$ represents the total number of the data sets, and $y_i$ represents the true value while $\hat{y_i}$ represents the estimated value.  
The lower the cost function of a model is, the more effective the classification of data is. So how to find the parameter to make the cost function reach its minimum is the problem worth solving.


### 2.3 Gradient Descent
To achieve the parameter which can make the cost function reaches its minimum, we need to introduce the gradient descent method to approach the minimum value of a function. More specifically, there are three different methods, each of which has its own form of recursive function as follows: $\quad$  
(1) Batch Gradient Descent  
The basic algorithm of BGD is as follows[1]:  
> Repeat until convergence$\{$ $$\omega_j:=\omega_j+\alpha\sum_{i=1}^{m}\left(y^{(i)}-h_{\omega}(x^{(i)})\right)x_j^{(i)}\quad\text{for every }j.$$
$\}$   

When the above equation converges to certain amount, that is to say the two consecutive iterations don't have a difference bigger than convergence, it quits the recursive loop. $\alpha$ is the learning rate, which determines the step of the descent.  
What "Batch" means here is that the output of each parameter $\theta$ within the parameter set $\omega$ should be calculated only when all the training data are covered.  
$\quad$  
(2) Stochastic Gradient Descent  
The basic algorithm of SGD is as follows[1]:   
> Loop$\{$  
$\quad$for $i=1$ to $m$ ,$\{$
$$\omega_j:=\omega_j+\alpha\sum_{i=1}^{m}\left(y^{(i)}-h_{\omega}(x^{(i)})\right)x_j^{(i)}\quad\text{for every }j.$$
$\quad\}$  
$\}$  

This method only calculated the parameters by considering the current training data, regardless of the other training data point, which will make the algorithm much more efficient. However, due to the inconsideration of other points, the recursive process will be quite turtuous, and most of time it can only approach the local best instead of reaching it. So it suits for the case where there are too many samples.  
$\quad$   

### 2.4 Text Feature Extraction
Since the object of our homework is to identify and classify emails, we need to extract the features of the text in the emails and evaluate the weights of each word. Thus we introduce the TF-IDF (term frequency–inverse document frequency) transform to process the email text provided, the basic calculation of which is as follows[2]: 
$$tf_{i,j}=\frac{n_{i,j}}{\sum_k n_{k,j}}\tag{2.7}$$
$$idf_i=\text{log} \frac{\left| D \right|}{\left |\{j:t_i \in d_j\}\right|}\tag{2.4}$$
$$tfidf_{i,j}= tf_{i,j} \times idf_i \tag{2.8}$$
Where $ n_{i,j}$ represents the times that term $t_i$ emerges in the file $d_j$ , while $\left| D \right|$ represents the total amount of the files in the database, and $\left |\{j:t_i \in d_j\}\right|$ represents the amount of the files that contain term $t_i$ .
So the tdidf value is the tool that we use to evaluate the importance of a certain term within the emails database, which can be used to judge whether a email is a spam or not. 
 
 
### 2.5 Regularization & 10-fold Cross Validation
To avoid over-fitting in the parameter calculation, we need to introduce a regularizer to amend the cost function. In most cases, we use $\lambda ||\omega||^2$ as the regularizer, which will make the cost function to be as follows[1]: 
$$ J(\omega)=\frac{1}{M}\sum_{i=1}^{M} \left( y_i \text{log}\hat{y_i} + (1-y_i) \text{log}(1-\hat{y_i}) \right)+ \frac{\lambda}{M} ||\omega||^2
\tag{2.9}$$
To get the optimum value of the regularization parameter $\lambda$ , we need to introduce the 10-fold cross validation method, which is the same as the first homework.



## 3. Spam Email Detector Realization


### 3.1 Batch Gradient Descent (Original Model)
The original basic method we designed to realize the spam email detector is as follows:    
$\quad$  
Text preprocess (Tokenize & Remove the stopwords and punctuations)  
Calculate and get the $x=\text{tf-idf}$ matrix  
For $\lambda = a$ to $b$  
$\quad$For 10-fold cross validation  
$\quad \quad$While cost function$>\epsilon$  
$\quad \quad \quad$For $i=1$ to $N$   
$\quad \quad \quad \quad$Update $\theta_i$ within parameter set $\omega$  
$\quad \quad \quad \quad$For $t=1$ to $M$  
$\quad \quad \quad \quad \quad$Calculate $\hat{y_i}$ and cost  
$\quad \quad \quad \quad$End  
$\quad \quad \quad$End  
$\quad$End  
Calculate the average loss of the 10 folds  
End  
Find the best $\lambda$  
$\quad$  
Where $N$ represents the total number of the valid terms within text data, while $M$ represents the total number of the training email amount each time.

#### 3.1.1 Simulation
The libraries we used to realize the whole function of the detector is as follows (The same to the next few amendments): 

In [1]:
import csv
import numpy
from numpy import *
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

The code to realize the text preprocess (Tokenize, Remove the stopwords and punctuations, Get the tf-idf matrix) function is as follows (The same to the next few amendments): 

In [ ]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
v = TfidfVectorizer(smooth_idf=False)
stop = stopwords.words('english')
sent_tokenizer = RegexpTokenizer(r'\w+')

# first part of homework3
my_list = []
csv_file = 'train.csv'
Processed_csv = 'new_train.csv'
new_file = open(Processed_csv , 'w')
fr = csv.writer(new_file, dialect="excel")

# we need to process the data in the train.csv first



def process(mail,label):
    if label == "ham" :
        new_label = 1
    else :
        new_label = 0

    mail = mail.lower()
    mail_tokenize = sent_tokenizer.tokenize(mail)
    filtered_words = [' '.join(w for w in mail_tokenize if not w in stop)]
    filtered_words.insert(0,new_label)
    return filtered_words


with open(csv_file, 'r', encoding='latin-1') as csv_reader:
    next(csv_reader)
    # we use csv_reader function to read documents from train.csv
    reader = csv.reader(csv_reader)
    my_list = list(reader)
    # we create two variables and save the content of mail into variable mail
    for row in my_list:
        mail = row[1]
        label = row[0]
        fr.writerow(process(mail,label))
new_file.close()
with open(Processed_csv, 'r', encoding='latin-1') as new_reader:
    new_train = csv.reader(new_reader)
    words = []
    for lines in new_train:
        words.append(lines[1])
words_Frequency = vectorizer.fit_transform(words)
words_bag = vectorizer.get_feature_names()
tf_idf = transformer.fit_transform(words_Frequency)

# normalize the tf-idf matrix
# get the maximum and minimum value
tf_idf_min, tf_idf_max = tf_idf.min(), tf_idf.max()
# (matrix elements-minimum)/(maximum-minimum)
a = (tf_idf - tf_idf_min) / (tf_idf_max - tf_idf_min)
words = []
labels = []
with open('new_train.csv', 'r') as new_csv_reader:
    reader = csv.reader(new_csv_reader)
    for line in reader:
        if line:
            words.append(line[1])
            labels += line[0]

error_sum = 0
lambda_value = []
y = numpy.matrix(labels).transpose().astype(int)
w = numpy.zeros((tf_idf.shape[1],1))
cross_validation = KFold(n_splits=10)

The code to realize the whole function of the detector using Batch Gradient Descent is as follows:

In [ ]:
class BatchGradient:
    def batch_gd(x, y, w, lam):
        w = numpy.zeros((x.shape[1], 1))      # w is matrix 6125 x 1 as temporary w
        error_sum = 0
        error0=0
        n = 1
        while True:
            z = (numpy.matmul(-x, w))  # matrix 2700 x 1   [2700x6125] x [6125x1]=2700x1
            # now set  n0 = 0.3
            learning_rate = 0.3 * (n ** (-0.9))
            for i in range(len(z)):  # i from (0,2700)
                # print("now is selecting   " + str(i) + " e-mail")
                y_tutor = 1 / (1 + math.exp(z[i]))
                dif = numpy.matrix((y_tutor - y[i][0]) * x[i, :]).transpose()
                error_sum += y[i] * math.log2(y_tutor) + (1 - y[i]) * math.log2(1 - y_tutor)
                for j in range(len(w)):  # j from (0,6125), done one e-mail than update one w
                    w[j][0] = w[j][0] - learning_rate * error_sum
            error = (1 / len(z)) * (error_sum + lam * numpy.matmul(w.transpose(), w))
            if abs(error-error0) < 0.1:
                break
            else:
                error0 = error
                w = w
                n+=1
                print(w)
        print("Done ,the best w is ", w)
        return error

#### 3.1.2 Result
The result of the above method is as follows:   
Error:11%
  (0, 0)	0.971364036086  
  (0, 1)	0.759948280032  
  (0, 2)	0.95585104881  
  (0, 3)	0.971364036086  
  (0, 4)	0.971364036086  
  (0, 5)	0.0599466459071  
  (0, 6)	0.971364036086  
  (0, 7)	0.971364036086  
  (0, 8)	0.971364036086  
  (0, 9)	0.971364036086  
  (0, 10)	0.971364036086  
  (0, 11)	0.971364036086  
  (0, 12)	0.0137686298225  
  (0, 13)	0.791249357017  
  (0, 14)	0.971364036086  
  (0, 15)	0.971364036086  
  (0, 16)	0.623288242111  
  (0, 17)	0.971364036086  
  (0, 18)	0.971364036086  
  (0, 19)	0.971364036086   
  (0, 20)	0.0279610823561   
  (0, 21)	0.198062010111  
  ...
  ...
  ...
  (0, 6144)	0.971364036086  
Computation Time: 4 hours 
  
### 3.2 Stochastic Gradient Descent 

#### 3.2.1 Simulation
The code to realize the whole function of the detector using Stochastic Gradient Descent is as follows:  


In [ ]:
class StochasticGradient:
    def stochastic_gd(tf_idf,y,w,lambda_1):
        # i don't know what the value of the epsilon
        # initial value
        w_value = numpy.zeros(tf_idf.shape[1],1)
        error_1 = 0
        error_0 = 0
        n = 1 # count number
        while True:
            h_w = (numpy.matmul(-tf_idf.toarray(),w))
            learning_rate = 0.3*(n ** (-0.9))
            print("this is the ", n , "times studying")
            print("=================================")
            for i in range(len(h_w))：
                y_tutor = 1 / (1+math.exp(h_w[i]))
                diverg = numpy.matrix((y_tutor-y[i][0]) * tf_idf[i, :]).transpose()
                for j in range(len(w_value)):
                    w_value[j][0] = w_value[j][0] - learning_rate * diverg[j][0]
            error = (1 / len(h_w)) * (error_1 + lambda_1 * numpy.matmul(w_value.transpose(), w_value))
            if abs(error-error_0) < 0.1:
                break
            else:
                error_0 = error
                w = w_value
                n +=1
                print(w)
        print("studying is over and the best w is ", w)
        return error

#### 3.2.2 Result & Comparision with BG
The stochastic gradient descent is more faster than BG, but it is more easier coverge at local optimal solution. 
  
  
### 3.3 Exponentiated Gradient Descent

#### 3.3.1 Simulation
The code to realize the the whole function of the detector using Exponentiated Gradient Descent is as follows[3]:

In [ ]:
class ExponentialGradient:
    def test_case_1(self):
        data_set = a.toarray()
        data_set_1 = numpy.array(data_set)
        self.weighted_vote_method(data_set_1)

    def weighted_vote_method(self, data_set, learning_rate = 0.3):
        if len(data_set) <= 0:
            raise ValueError("Data set length error.")
        weight_result =  []
        current_weight = [1. / len(data_set[0,:]) for i in range(len(data_set[0,:]))]
        for i in range(len(data_set[0])):
            #print("Current weight=\t\t" + str(current_weight))
            current_weight_plus = current_weight
            current_weight_minus = current_weight
            current_weight = self.exponentiated_gradient(data_set[i,:], current_weight_plus, current_weight_minus, learning_rate)
            weight_result.append(current_weight)
            #print("===================")
    def exponentiated_gradient(self, data_set, previous_weight_plus, previous_weight_minus, learning_rate):
        if len(data_set) <= 0:
            raise ValueError("Data set length error.")
        if len(data_set) != len((previous_weight_plus-previous_weight_minus):
            raise ValueError("Arguments length not equal.")

        #print("Data set =\t\t" + str(data_set))

        result = []
        all_weighted_value = numpy.sum([previous_weight[i] * data_set[i] for i in range(len(data_set))])
        # update the w+ and w-
        numerator_plus = numpy.sum([previous_weight_plus[i] * numpy.exp((learning_rate * data_set[i]) / all_weighted_value) for i in range(len(data_set))])
        numerator_minus = numpy.sum([previous_weight_minus[i] * numpy.exp(-(learning_rate * data_set[i]) / all_weighted_value) for i in range(len(data_set))])
        #print("Numerator=\t\t\t" + str(numerator))

        for i in range(len(data_set)):
            fractions = (previous_weight_plus[i]-previous_weight_minus[i]) * numpy.exp((learning_rate * data_set[i]) / all_weighted_value)
            result.append(fractions / (numerator_plus-numerator_minus)
        #print("Result=\t\t\t\t" + str(result))
        return result

#### 3.3.2 Result & Comparision with BG
The result of the above method is as follows:  
Final Cost:  
Computation Time:  
  

### 3.4 Additional Work

We change the regularizer in the function (2.9) to $\lambda||\omega||$ . And we can get the following results:  

In [ ]:
class BatchGradient:
    def batch_gd(x, y, w, l):
        w = numpy.zeros((x.shape[1], 1))      # w is matrix 6125 x 1 as temporary w
        error_sum = 0
        error0=0
        n = 1
        while True:
            z = (numpy.matmul(-x, w))  
            # now set  n0 = 0.3
            learning_rate = 0.3 * (n ** (-0.9))
            for i in range(len(z)):  # i from (0,2700)
                y_tutor = 1 / (1 + math.exp(z[i]))
                dif = numpy.matrix((y_tutor - y[i][0]) * x[i, :]).transpose()
                error_sum += y[i] * math.log2(y_tutor) + (1 - y[i]) * math.log2(1 - y_tutor)
                for j in range(len(w)):  # j from (0,6125), done one e-mail than update one w
                    w[j][0] = w[j][0] - learning_rate * dif[j][0]
            error = (1 / len(z)) * (error_sum + l * numpy.matmul(w.transpose(), w))
            if abs(error-error0) < 0.1:
                break
            else:
                error0 = error
                w = w
                n+=1
                print(w)
        print("Done ,the best w is ", w)
        return error

## 4. Conclusion

In this homework, we use logistic regression as tools to handle the problem of spam email detection. The method we use have batch gradient descent, stochastic gradient descent and exponential gradient descent. The simulation results show that the logistic regression can effectively classify the spam email and ham email, but different methods have different computation time and convergence speed. In general, because the batch gradient descent need to cover all data in train set so it is cost large time on computation and convergence speed.With the regularization term $\lambda$, we can effectively avoid the problem of overfitting and we try different regularization term like $\lambda ||\omega||$, but because of time problem, we can not try every method like weighted least linear square and comparsion.

## 5.Reference
1. http://www.cnblogs.com/rcfeng/p/3958926.html 
2. http://blog.csdn.net/sangyongjia/article/details/52440063
3. Kivinen, Jyrki, and Manfred K. Warmuth. "Exponentiated gradient versus gradient descent for linear predictors." Information and Computation 132.1 (1997): 1-63.